In [23]:
# Loading Required Packages 
import cobra
from cobra.flux_analysis import flux_variability_analysis
import pandas as pd
import numpy as np
import itertools

# Loading the Model 
model = cobra.io.read_sbml_model("ArabidopsisCoreModel1.xml")

# Changing the directionality of Fum_c to Mal_c to go the other way (day-time model) 
r = model.reactions.get_by_id('FumHA_c')
r.add_metabolites({model.metabolites.get_by_id('Fum_c'): +2})
r.add_metabolites({model.metabolites.get_by_id('H2O_c'): +2})
r.add_metabolites({model.metabolites.get_by_id('Mal_c'): -2})
print(r.reaction)
print(r.bounds)
print("Directionality changed. Dyson et al. 2016 \n")

# Creating a Malate "Storage" Reactions
r = cobra.Reaction('Mal_Store')
r.name = 'Mal_Store'
r.add_metabolites({model.metabolites.get_by_id('Mal_c'): -1})
model.add_reaction(r)
print(r.reaction)

# Creating a Fumarate "Storage" Reactions
r = cobra.Reaction('Fum_Store')
r.name = 'Fum_Store'
r.add_metabolites({model.metabolites.get_by_id('Fum_c'): -1})
model.add_reaction(r)
print(r.reaction)

# Creating a Starch "Storage" Reactions
r = cobra.Reaction('Starch_Store')
r.name = 'Starch_Store'
r.add_metabolites({model.metabolites.get_by_id('starch1_h'): -1})
model.add_reaction(r)
print(r.reaction)


cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '2*(ATCG00020 and ATCG00680 and ATCG00280 and ATCG00270 and ATCG00580 and ATCG00570 and ATCG00710 and ATCG00080 and ATCG00550 and ATCG00070 and ATCG00560 and ATCG00220 and ATCG00700 and (AT5G66570 or AT3G50820) and AT1G06680 and (AT4G21280 or AT4G05180) and AT1G79040 and AT1G44575 and ATCG00690 and AT3G21055 and AT2G30570 and AT2G06520 and AT1G67740 and ATCG00300)' for <Reaction PSII_h at 0x7f74543a7ed0>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '2*(ATCG00540 and ATCG00720 and AT4G03280 and ATCG00730 and ATCG00600 and ATCG00590 and AT2G26500 and ATCG00210)' for <Reaction Cytb6f_h at 0x7f74543a7e90>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '8*(ATCG00490) and 8*(AT5G38430 or AT5G38420 or AT1G67090 or AT5G38410)' for <Reaction RBC_h at 0x7f74543ad310>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '4*(AT3G26650 or AT1G12900) or (2*(AT3G26650 or 

Mal_c --> Fum_c + H2O_c
(0.0, 1000.0)
Directionality changed. Dyson et al. 2016 

Mal_c --> 
Fum_c --> 
starch1_h --> 


In [24]:
# Printing all import and export reactions in the model
def input_output_reacs(model):
    "Prints all input and output reactions for the given model"
    reacs = []
    for r in model.reactions:
        coeffs = []
        for i in r.metabolites:
            c = r.get_coefficient(i.id)
            coeffs.append(c)
        if all(i > 0.0 for i in coeffs) or all(i < 0.0 for i in coeffs) :
            reacs.append(r)
    return(reacs)

exp_reacs = input_output_reacs(model)

In [25]:
# Appending transport reactions which are not to be deleted
r1 = model.reactions.get_by_id("Tr_TPT1")
r2 = model.reactions.get_by_id("Tr_TPT2")

r = r1
r.upper_bound = 0.0
r.lower_bound = 0.0


# Setting reactions without AT codes to zero
noAT = []
for r in model.reactions:
    g = r.genes
    for gen in g:
        if gen.id == '-':
            if r not in exp_reacs:
#                 if r.id[0] is not "T":
#                     print(r.id)
#                     print(r.reaction)
#                     r.upper_bound = 0.0
#                     r.lower_bound = 0.0
                noAT.append(r)
print(len(noAT))


IPODC_h
H_h + IPO_h --> 4MOP_h + CO2_h
GluSeADA_c
Glu_DASH_SeA_c --> H2O_c + H_c + P5C_c
GluSeADA_h
Glu_DASH_SeA_h --> H2O_h + H_h + P5C_h
GluSeADA_m
Glu_DASH_SeA_m --> H2O_m + H_m + P5C_m
Im_Pi
3.0 ATP_c + 3.0 H2O_c --> 3.0 ADP_c + 2.0 H_h + 3.0 Pi_c + Pi_h
Im_NO3
2.0 ATP_c + 2.0 H2O_c --> 2.0 ADP_c + 2.0 H_c + NO3_c + 2.0 Pi_c
Im_SO4
3.0 ATP_c + 3.0 H2O_c --> 3.0 ADP_c + 3.0 H_c + 3.0 Pi_c + SO4_c
Ex_MACP
M_DASH_ACP_h --> ACP_h + H_h
NGAM_c
ATP_c + H2O_c --> ADP_c + H_c + Pi_c
NGAM_h
ATP_h + H2O_h --> ADP_h + H_h + Pi_h
NGAM_m
ATP_m + H2O_m --> ADP_m + H_m + Pi_m
Bio_CLim
0.58559 ATP_c + 207.46087 Ala_c + 64.19766 Arg_c + 61.82037 Asn_c + 90.68407 Asp_c + 0.49638 CTP_c + 27.91206 Cys_c + 14.77415 Frc_c + 77.58391 Fum_m + 0.15305 GABA_c + 0.40478 GTP_c + 33.84942 Glc_c + 45.7267 Gln_c + 103.37619 Glu_c + 245.76971 Gly_c + 29.81972 His_c + 80.57171 Ile_c + 142.42163 Leu_c + 92.89756 Lys_c + 779.97036 M_DASH_ACP_h + 38.11199 Mal_m + 2.27742 Mas_c + 30.37981 Met_c + 14.60727 Orn_h + 59.6

In [26]:
# Get list of all possible proteins in the model
gene_ids = []
for g in model.genes:
    gene_ids.append(g.id)

In [27]:
# Reading in the protein data
df = pd.read_excel(io="ProteinConc.xlsx")

In [28]:
# Get list of all measured proteins 
all_atg = []
x=df["Accession"]
for atg in x.values:
    all_atg.append(atg)

In [29]:
# Comparing the measured protein concentrations to those which are in the model 

model_atg = [] # Total measured proteins in model
model_atg_final = [] # Proteins with direct reaction constraint (i.e. single responsible protein)
model_rxns_atg = [] # Model reactions with direct constraints (i.e. single responsible protein)
model_rxns_atgs = [] # Model reactions with multiple possible constraints 
model_atgs_final = [] # Proteins with multiple possible constraints 

for atg in x.values: # Check if protein is in the model 
    if atg in gene_ids:
        model_atg.append(atg)
        # Only parameterize that reaction if that enzyme is the only one responsible for that reaction
        for r in model.genes.get_by_id(atg).reactions:
            if len(r.genes) == 1:
                model_rxns_atg.append(r)
                model_atg_final.append(atg)
            else:
                prot_names = []
                for g in r.genes:
                    prot_names.append(g.id)
                if set(prot_names).issubset(set(all_atg)):
                    model_rxns_atgs.append(r)
                    model_atgs_final = model_atgs_final + prot_names

# Feasible constraint summary 
# General 
print("Total measured proteins: {}".format(len(all_atg)))  
print("Total measured proteins in model: {}".format(len(model_atg))) 
print("Total model reactions: {}".format(len(model.reactions)))
print(" ")
# Reactions with simple constraints 
print("Total model reactions with single constraints: {}".format(len(set(model_rxns_atg))))
print("Model proteins for single constraints: {}".format(len(set(model_atg_final))))
print(" ")
# Reactions with multiple possible constraints 
print("Total model reactions with multiple constraints: {}".format(len(set(model_rxns_atgs))))
print("Total model proteins for possible constraints: {}".format(len(set(model_atgs_final))))

Total measured proteins: 2427
Total measured proteins in model: 245
Total model reactions: 552
 
Total model reactions with single constraints: 62
Model proteins for single constraints: 49
 
Total model reactions with multiple constraints: 39
Total model proteins for possible constraints: 53


In [30]:
constr_reacs = set(model_rxns_atg + model_rxns_atgs)
constr_ids = []
for r in constr_reacs:
    constr_ids.append(r.id)

In [31]:
# Function for constraining the model reactions for a specific genotype and temperature
def constr(reacs,Dat1,Dat2,Dat3,Dat4):
    for r in reacs:
        upper_vals = []
        lower_vals = []
        for g in r.genes:
            # Get index of protein in data file 
            ind = all_atg.index(g.id)
            # Scaling the values for setting fluxes 
            vals = [i/4000. for i in Dat1[ind],Dat2[ind],Dat3[ind],Dat4[ind]]
            val = np.mean(vals)
            sterr = np.std(vals)/np.sqrt(len(vals))
            upper = val + sterr
            lower = val - sterr
            upper_vals.append(upper)
            lower_vals.append(lower)
        # Constraint flux of forward reaction
        r.upper_bound = round(sum(upper_vals),3)
        # Constraint flux of lower reaction 
        if r.lower_bound < 0.0:
            r.lower_bound = round(-sum(upper_vals),3)
        # No lower bound b/c don't know localization or enzymes are used to in other reactions 

In [32]:
# RUN THIS NEXT SECTION AS EITHER OR! RESERT KERNEL EACH TIME. 

rMal = model.reactions.get_by_id('Mal_Store')
rFum = model.reactions.get_by_id('Fum_Store')
rStarch = model.reactions.get_by_id('Starch_Store')
rCO2 = model.reactions.get_by_id('Im_CO2')
FumHA_c = model.reactions.get_by_id('FumHA_c')
NADPH = model.reactions.get_by_id('Fd_DASH_NADPR_h')
ATPase = model.reactions.get_by_id('ATPase_h')
all_reac = [rMal,rFum,rStarch,rCO2,FumHA_c,NADPH,ATPase]
rObj = model.reactions.get_by_id('Im_CO2')
print(rObj.reaction)
model.objective = rObj
model.objective_direction = "Maximum"
print(model.objective)

f1=0.99*200
f2=1.01*200

 --> CO2_c
Maximize
-1.0*Im_CO2_reverse_a674e + 1.0*Im_CO2


In [33]:
# Parameterizing the model for Col-0 Control Conditions

# Col0 Control
rCO2.upper_bound = 101.0
rCO2.lower_bound = 99.0
rMal.upper_bound = 0.004*f2
rMal.lower_bound = 0.002*f1
rFum.upper_bound = 0.007*f2
rFum.lower_bound = 0.005*f1
rStarch.upper_bound = 0.029*f2
rStarch.lower_bound = 0.027*f1

# Set up reactions which will be constrained
constr_reacs = set(model_rxns_atg + model_rxns_atgs)

# Constraining the model
constr(constr_reacs,df["WT Cntl 1"],df["WT Cntl 2"],df["WT Cntl 3"],df["WT Cntl 4"])

print(cobra.flux_analysis.flux_variability_analysis(model, all_reac))
# Saving the model
#cobra.io.write_sbml_model(model,"Arnold2014_WT_Cntl_All_removeAT.xml",use_fbc_package=False)

                    maximum     minimum
Mal_Store          0.808000    0.396000
Fum_Store          1.414000    0.990000
Starch_Store       5.858000    5.346000
Im_CO2           101.000000  101.000000
FumHA_c            1.414000    0.000000
Fd_DASH_NADPR_h  250.000000  141.873429
ATPase_h         107.142857   85.551000


In [ ]:
# Parameterizing the model for Fum2 Control Conditions

# Fum2 Control
rCO2.upper_bound = 101.0
rCO2.lower_bound = 99.0
rMal.upper_bound = 0.013*f2
rMal.lower_bound = 0.011*f1
rFum.upper_bound = 0.0*f2
rFum.lower_bound = 0.0*f1
rStarch.upper_bound = 0.043*f2
rStarch.lower_bound = 0.041*f1

# Set up reactions which will be constrained
constr_reacs = set(model_rxns_atg + model_rxns_atgs)

# Constraining the model
r = model.reactions.get_by_id('FumHA_c')
r.upper_bound = 0.0 # Mutant knockout 
r.lower_bound = 0.0 
constr(constr_reacs,df["fum2 Cntl 1"],df["fum2 Cntl 2"],df["fum2 Cntl 3"],df["fum2 Cntl 4"]) #Proteomics 

print(cobra.flux_analysis.flux_variability_analysis(model, all_reac))
# Saving the model
#cobra.io.write_sbml_model(model,"Arnold2014_M_Cntl_All_removeAT.xml",use_fbc_package=False)

In [ ]:
# Parameterizing the model for Col-0 Cold Conditions

# Col0 Cold
rCO2.upper_bound = 101.0
rCO2.lower_bound = 99.0
rMal.upper_bound = 0.013*f2
rMal.lower_bound = 0.011*f1
rFum.upper_bound = 0.024*f2
rFum.lower_bound = 0.022*f1
rStarch.upper_bound = 0.052*f2
rStarch.lower_bound = 0.05*f1

# Set up reactions which will be constrained
constr_reacs = set(model_rxns_atg + model_rxns_atgs)

# Constraining the model
constr(constr_reacs,df["WT cold 1"],df["WT cold 2"],df["WT cold 3"],df["WT cold 4"])

print(cobra.flux_analysis.flux_variability_analysis(model, all_reac))
# Saving the model
#cobra.io.write_sbml_model(model,"Arnold2014_WT_Cold_All.xml",use_fbc_package=False)

In [ ]:
# Parameterizing the model for Fum2 Cold Conditions

# Fum2 Cold
rCO2.upper_bound = 95.0
rCO2.lower_bound = 93.0
rMal.upper_bound = 0.026*f2
rMal.lower_bound = 0.024*f1
rFum.upper_bound = 0.0*f2
rFum.lower_bound = 0.0*f1
rStarch.upper_bound = 0.064*f2
rStarch.lower_bound = 0.062*f1

# Set up reactions which will be constrained
constr_reacs = set(model_rxns_atg + model_rxns_atgs)

# Constraining the model
r = model.reactions.get_by_id('FumHA_c')
r.upper_bound = 0.0 # Mutant knockout 
r.lower_bound = 0.0 
constr(constr_reacs,df["fum2 Cold 1"],df["fum2 Cold 2"],df["fum2 Cold 3"],df["fum2 Cold 4"]) #Proteomics 

print(cobra.flux_analysis.flux_variability_analysis(model, all_reac))
# Saving the model
#cobra.io.write_sbml_model(model,"Arnold2014_M_Cold_All.xml",use_fbc_package=False)

In [ ]:
pathwayA = ["Tr_TPT1"]
pathwayB = ["PGAK_h","GAPDH2_h","Tr_TPT2","GAPDH3_c"]
both = ["RBC_h","PGAM_c","Enol_c","PEPC2_c","MalDH1_c","FumHA_c"]
allnames = both+pathwayB+pathwayA
for rname in allnames:
    r=model.reactions.get_by_id(rname)
    print(r.id)
    print(r.reaction)
    all_reac.append(rname)


rObj = model.reactions.get_by_id('Im_CO2')
print(rObj.reaction)
model.objective = rObj
model.objective_direction = "Maximum"
print(model.objective)
print(cobra.flux_analysis.flux_variability_analysis(model, all_reac))

rObj = model.reactions.get_by_id('ATPase_h')
print(rObj.reaction)
model.objective = rObj
model.objective_direction = "Minimum"
print(model.objective)
print(cobra.flux_analysis.flux_variability_analysis(model, all_reac))

rObj = model.reactions.get_by_id('Fd_DASH_NADPR_h')
print(rObj.reaction)
model.objective = rObj
model.objective_direction = "Minimum"
print(model.objective)
print(cobra.flux_analysis.flux_variability_analysis(model, all_reac))

In [ ]:
# CHRR flux sampling (MATLAB SCRIPT)
# Plotting the overlapping sampling distribution (NEW PYTHON SCRIPT)

In [ ]:
# Deleting model reactions as sets 
req_reacs = []
Nreq_reacs = []

# Exluding reactions which cannot be deleted in combination with others
# Round 1
exp_reacs.append(model.reactions.get_by_id("Tr_O2h"))
# Round 2 
exp_reacs.append(model.reactions.get_by_id("Tr_CO2h"))
# Round 3
exp_reacs.append(model.reactions.get_by_id("Tr_H2Oh"))
exp_reacs.append(model.reactions.get_by_id("Im_NO3"))
exp_reacs.append(model.reactions.get_by_id("Tr_O2m"))
exp_reacs.append(model.reactions.get_by_id("Tr_H2Oh"))
exp_reacs.append(model.reactions.get_by_id("Tr_O2m"))
exp_reacs.append(model.reactions.get_by_id("Tr_NO2"))
exp_reacs.append(model.reactions.get_by_id("Tr_NTT"))
exp_reacs.append(model.reactions.get_by_id("Tr_AAC"))
exp_reacs.append(model.reactions.get_by_id("Tr_TPT1"))
exp_reacs.append(model.reactions.get_by_id("Tr_TPT2"))
exp_reacs.append(model.reactions.get_by_id("Tr_TPT3"))

noAT = []
for r in model.reactions:
    g = r.genes
    for gen in g:
        if gen.id == '-':
            if r not in exp_reacs:
                noAT.append(r)
                r.upper_bound = 0.0
                r.lower_bound = 0.0
                
print(len(noAT))

print(cobra.flux_analysis.flux_variability_analysis(model, all_reac))

# # Trying to delete as high a set of them as possible 
# for s in set(itertools.combinations(noAT, 4)): #set(itertools.combinations(noAT, 1))
#     print("Check")
#     b = []
#     for i in range(len(s)):
#         b.append(s[i].bounds)
#         s[i].upper_bound = 0.0
#         s[i].lower_bound = 0.0
#     interest_reacs = ["Im_CO2","Mal_Store","Fum_Store","Starch_Store","Ex_Suc"]
#     test = []
#     for r in interest_reacs:
#         model.objective = r
#         try:
#             bounds = flux_variability_analysis(model, r)
#             if bounds['minimum'][0] > 0.0:
#                 test.append(0) # Carbon storage ok
#             else:
#                 test.append(1)
#         except Exception:
#             test.append(1)
#     if sum(test) > 0: # Can't store carbon 
#         req_reacs.append(s)
#     else:
#         Nreq_reacs.append(s)
#     for i in range(len(s)):
#         # Resetting the reactions bounds
#         s[i].upper_bound = b[i][1]
#         s[i].lower_bound = b[i][0]
# print("Finished")

In [ ]:
# Deleting model reactions as sets 
req_reacs = []
Nreq_reacs = []

# Trying to delete as high a set of them as possible 
for s in set(itertools.combinations(model.reactions, 1)): #set(itertools.combinations(noAT, 1))
    b = []
    for i in range(len(s)):
        b.append(s[i].bounds)
        s[i].upper_bound = 0.0
        s[i].lower_bound = 0.0
    interest_reacs = ["Im_CO2","Mal_Store","Starch_Store","Ex_Suc"]
    test = []
    for r in interest_reacs:
        model.objective = r
        try:
            bounds = flux_variability_analysis(model, r)
            if bounds['minimum'][0] > 0.0:
                test.append(0) # Carbon storage ok
            else:
                test.append(1)
        except Exception:
            test.append(1)
    if sum(test) > 0: # Can't store carbon 
        req_reacs.append(s)
    else:
        Nreq_reacs.append(s)
    for i in range(len(s)):
        # Resetting the reactions bounds
        s[i].upper_bound = b[i][1]
        s[i].lower_bound = b[i][0]